In [1]:
import os
os.chdir('../')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_columns', None)
import seaborn as sns

In [3]:
indir = Path('drive/output/derived/project_outcomes')

In [4]:
time_period = 6
rolling_window = 732
threshold_stat = 0
post_periods = 2

In [5]:
for criteria_col in ['commits','comments']:
    for criteria_pct in [75, 90]:
        for consecutive_periods in [2, 3, 4, 5, 6]:
            try:
                df_departed = pd.read_parquet(f'drive/output/derived/contributor_stats/departed_contributors/departed_contributors_major_months{time_period}_window{rolling_window}D_criteria_{criteria_col}_{criteria_pct}pct_general25pct_consecutive{consecutive_periods}_post_period{post_periods}_threshold_gap_qty_{threshold_stat}.parquet')
                df_departed['diff'] = df_departed.groupby(['repo_name','actor_id'])['consecutive_periods'].transform('diff')
                df_departed = pd.merge(df_departed, df_departed.query('diff>1')[['repo_name','actor_id']].assign(drop = 1),
                         how = 'left').query('drop.isna()')
                contribution_histories = pd.read_parquet(f'drive/output/derived/contributor_stats/contributor_data/major_contributors_major_months{time_period}_window{rolling_window}D_samplefull.parquet')
                criteria_pct_cols = [f"{criteria_col}_{criteria_pct}th_pct",f"general_{criteria_col}_25th_pct"]
                df_pct = contribution_histories[['repo_name','time_period'] + criteria_pct_cols].drop_duplicates(subset = ['repo_name','time_period'])\
                    .sort_values(['repo_name','time_period'])
                departed_col_ordering = df_departed.columns
                df_departed = pd.merge(df_departed.drop(criteria_pct_cols, axis =1), df_pct, how = 'left')[departed_col_ordering]\
                    .query('below_qty_mean_gap0 == 1 | below_qty_mean_gap1 == 1')
                num_contributors = df_departed[['repo_name','actor_id']].drop_duplicates().shape[0]
                print(criteria_col, criteria_pct, consecutive_periods, rolling_window, post_periods, num_contributors) 
            except Exception as e:
                continue

In [6]:
time_period = 6
rolling_window = 732
criteria_col = 'comments'
criteria_pct = 75
consecutive_periods = 4
post_periods = 2
threshold_stat = 0
df_departed = pd.read_parquet(f'/scratch/chrisliao/oss_hierarchy/datastore/output/derived/contributor_stats/departed_contributors/departed_contributors_major_months{time_period}_window{rolling_window}D_criteria_{criteria_col}_{criteria_pct}pct_general25pct_consecutive{consecutive_periods}_post_period{post_periods}_threshold_gap_qty_{threshold_stat}.parquet')

PermissionError: [Errno 13] Permission denied: '/scratch/chrisliao/oss_hierarchy/datastore/output/derived/contributor_stats/departed_contributors/departed_contributors_major_months6_window732D_criteria_comments_75pct_general25pct_consecutive4_post_period2_threshold_gap_qty_0.parquet'